# CMI20

Atualmente vivemos uma revolução financeira. A politica Keynesiana vigente nas últimas décadas parece ter desgastado a fé pública nas moedas fiduciárias, e as criptomoedas tiraram grande proveito deste fato: estas aparentam apresentar todos os benefícios do sistema antigo, o ouro, sem nenhuma das disvatagens. Elas ainda trazem todo o benefício da tecnologia blockchain, um sistema robusto e transparente no qual seu "ouro criptográfico" não pode ser facilmente desviado. O volume total de capitalização do mercado de criptomoedas alcançou quinhentos bilhões de dólares em dezembro de 2017, com um aumento de dez vezes neste valor em apenas doze meses. Além disso a tecnologia esta evoluindo rapidamente, sendo que o blockchain original, sendo um caso particular de grafo acíclico direcionado (ou DAG, para directed acyclic graph), já é essencialmente obsoleto, e métodos muito mais eficiêntes e flexíveis vem tomando frente. Tecnologias que proporcionam transações completamente privadas e livre de taxações também oferecem grandes vantagens ao usuário.

Enquanto a Bitcoin ainda domina o cenário, existem inúmeras outras criptomoedas, que utilizam de novos métodos e tecnologias,e novas moedas são criadas todos os dias. Um índice de criptomoedas serve como referência para este mercado altamente dinâmico. A idéia do índice é proporcionar algo parecido com o índice Bovespa, ou o S&P 500 para o mercado americano, porém para criptomoedas. Escolhe-se um número significativo de integrantes do mercado (criptomoedas) com maior capitalização relativa, atribui-se pesos a estes integrantes e usa-se estes pesos para combinar seus preços em um índice único.



In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
import bokeh
from bokeh.io import output_notebook
from bokeh.layouts import column
from bokeh.palettes import inferno
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, Legend, Span, Label
output_notebook()

import tqdm

In [ ]:
start_date = '2015-01-01'
reevaluate_freq = 7
n = 20

data = pd.read_csv("../input/crypto-markets.csv")
marketcapDf = pd.pivot_table(data, index='date', columns='symbol', values='market',
                              fill_value=0.0).loc[start_date:]
marketcapDf = marketcapDf.apply(lambda x: [x[i] / x.sum() for i in range(x.shape[0])],
                                  axis=1).astype('f')
closeDf = pd.pivot_table(data, index='date', columns='symbol', values='close').astype('f').fillna(0).loc[start_date:]

In [ ]:
words = {}
for symbol in data.symbol.unique():
    words[symbol] = data.query("symbol == '%s'" % symbol).market.iloc[-1]

In [ ]:
from wordcloud import WordCloud
 
wc = WordCloud(width=1024, height=500).generate_from_frequencies(words, 400);
 
plt.figure(figsize=(12,12));
plt.imshow(wc, interpolation='bilinear');
plt.tight_layout(pad=4);
plt.axis("off");

### Sobre o CMI20

Como calculamos os integrantes do índice?
O volume de capitalização das criptomoedas é uma medida bastante volátil, e por isso os "top 20" são computados muito mais frequentemente do que, digamos, os integrantes do S&P 500. Decidimos por realizar um cálculo completo todo mês, e realizar um rebalanceamento nos componentes semanalmente. Adicionalmente, a volatilidade do espaço faz com que seja inviável calcular os maiores integrantes do mercado com base em uma medida pontual, digamos o valor de mercado no dia em questão. Para sanar este problema, utilizamos uma media ponderada exponencial dos valores de mercado durante o período entre cálculos como medida de capitalização.

### Como atribuímos pesos?

O S&P 500 pesa seus constituíntes de acordo com seus respectivos valores de mercado. Isto é (de certa forma) razoável para um mercado já maduro de equidades - de certa forma, pois os maiores 52 componentes deste índice totalizam 50% do peso total. A situação é muito mais extrema no espaço de criptomoedas, onde, atualmente, uma única integrante -- a Bitcoin -- representa aproximadamente 35% do total de capitalização do mercado. Isto faz com que um índice com pesos proporcionais não seja tão útil, então decidimos ponderar cada componente proporcionalmente a raíz quadrada de sua preponderâcia (suavizada).

In [ ]:
plt.figure(figsize=(13,13))
plt.pie([float(value) for value in words.values()], labels=list(words.keys()),  autopct='%1.1f%%');

### Como calculamos o índice?

Entre datas de rebalanceamento, o valor do índice é definido como:

$$ I_{t} = \sum_{j=1}^{20} W_{j} \frac{P_{j}(t)}{P_{j}(0)} $$

Onde $ I_{t} $ é o valor do índice no tempo $ t $ e $ W_{j} $ e $ P_{j} $ representam o peso e o preço atribuídos ao $ j-ésimo $ elemento do índice, respectivamente.  
Em datas de rebalanceamento, os pesos são normalizados de forma que que índice permaneça o mesmo, independente deste ser computado com os pesos antigos ou com os novos.  
O índice é calculado em tempo real. Todos os valores se referem ao preço de fechamento do dia anterior, considerado como sendo em 0000 GMT.

In [ ]:
plot = False

smoothed_mc = marketcapDf.ewm(alpha=0.05).mean()

if plot:
    mc_fig = figure(title="Preponderância media",
                   x_axis_type="datetime",
                   x_axis_label='date',
                   y_axis_label='Capital',
                   plot_width=900, plot_height=500,
                   tools=['crosshair','reset','xwheel_zoom','pan,box_zoom', 'save'],
                   toolbar_location="above"
               );

    palette = inferno(256)
    index = pd.to_datetime(smoothed_mc.index)
    for i, s in tqdm.tqdm((enumerate(smoothed_mc.columns))):
        mc_fig.line(
            index,
            smoothed_mc.iloc[:, i].values,
            color=palette[i % 255]
        );

    show(mc_fig);

In [ ]:
# Select largest marketcap coins
largest = pd.DataFrame(index=smoothed_mc.columns)
for i, row in enumerate(smoothed_mc.iterrows()):
    # choose pairs every quarter
    if i % int(reevaluate_freq) == 0:
        components_index = np.argpartition(row[1].values, -n)[-n:]
    
    largest[row[0]] = row[1].iloc[components_index].fillna(0)
largest = largest.T
largest.shape

In [ ]:
for col in largest.columns:
    if largest[col].isnull().all():
        largest = largest.drop(col, axis=1)
        closeDf = closeDf.drop(col, axis=1)
retDf = closeDf.fillna(0).rolling(2).apply(lambda x: np.clip(x[-1] / x[-2], 0, 2)).fillna(1).replace([np.inf, -np.inf], 1)
largest.shape, closeDf.shape, retDf.shape

In [ ]:
# Relative strength
plt.figure(figsize=(13,9))
for col in largest.columns:
    if col in list(largest.iloc[-1].dropna().index):
        plt.plot(pd.to_datetime(largest.index), largest[col].values, label=col)
    else:
        plt.plot(pd.to_datetime(largest.index), largest[col].values)
plt.legend()
plt.title("Força relativa")
plt.xlabel("tempo")
plt.ylabel("% mercado");

In [ ]:
# Daily rebalancing
sqrtDom = largest.fillna(0).apply(lambda x: np.sqrt(x) / np.sqrt(x).sum(), axis=1).replace([np.inf, -np.inf], 1)

plt.figure(figsize=(13,9))
for col in sqrtDom.columns:
    if col in list(largest.iloc[-1].dropna().index):
        plt.plot(pd.to_datetime(sqrtDom.index), sqrtDom[col].values, label=col)
    else:
        plt.plot(pd.to_datetime(sqrtDom.index), sqrtDom[col].values)
plt.legend()
plt.title("Raiz da força relativa")
plt.xlabel("tempo")
plt.ylabel("% mercado");

In [ ]:
def rebalance(oldWeights, relativeStrength, mpc=.1):
    # Optimization constraints
    constraints = [
        # analitycal value
#         {'type': 'eq', 'fun': lambda w: np.dot(oldWeights, close) - np.dot(w, close)},
        # Simplex constraints
        {'type': 'eq', 'fun': lambda w: w.sum() - 1},
    ]

    bounds = []
    for item in relativeStrength:
        if not np.allclose(0.0, item, rtol=1e-8, atol=1e-8):
            bounds.append((0, mpc))
        else:
            bounds.append((0,0))
    
    b = minimize(
        lambda w: np.linalg.norm(relativeStrength - w),
        relativeStrength,
        tol=None,
        constraints=constraints,
        bounds=bounds,
        jac=False,
        method='SLSQP',
        options = {'disp': False,
                   'iprint': 1,
                   'eps': 1.4901161193847656e-08,
                   'maxiter': 666,
                   'ftol': 1e-12
                   }
    )
    
    return b['x'], b['fun'], b['nit']

In [ ]:
cmi20 = []
weights = [sqrtDom.iloc[0].values]
# rebalance portfolio every day
for i, row in enumerate(sqrtDom.iterrows()):
    ret = retDf.iloc[i].values
    relativeStrength = row[1].values
    # recalculate weights every quarter
    if i % int(reevaluate_freq) == 0:
        w, loss, niter = rebalance(weights[-1], relativeStrength)
        print(i, loss, niter)
        weights.append(w)
        
    cmi20.append((largest.index[i], np.dot(weights[-1], ret)))
    
cmi20 = pd.DataFrame.from_records(cmi20).set_index(0, drop=True).cumprod()

In [ ]:
# Index weights over time
plt.figure(figsize=(13,6))
plt.subplot(121)
plt.plot(weights[1:])
plt.title("Distribuição histrica")
plt.ylabel("% portfolio")
plt.xlabel("rebalanços")
plt.grid()

cols = sqrtDom.iloc[-1].notnull()
symbols = sqrtDom.iloc[-1][cols]
symbols = list(symbols.index)
w = pd.Series(index=sqrtDom.columns, data=weights[-1])[symbols].sort_values()

plt.subplot(122)
plt.pie(w, labels=w.index,  autopct='%1.1f%%')
plt.title("Distribuição atual");

Um portfólio composto dos maiores integrantes em capital de mercado se propõe a descrever a média da movimentação do setor, como demonstrado na análise:

In [ ]:
# marketcap captured by the cmi20 index
plt.figure(figsize=(13,7))
largest.fillna(0).sum(axis=1).plot()
plt.title("Parcela do capital descrita")
plt.xlabel("tempo")
plt.ylabel("% mercado");

### Os Resultados

Não surpreendentemente, o índice constitui um veículo de investimento muito melhor do que o Bitcoin por si. Nós análisamos o desempenho do CMI20 desde o início de 2015 até o inicio de 2018. A Bitcoin teve um crescimento de aproximadamente 34 vezes, apresentando uma grande volatilidade. O índice das 20 maiores moedas obteve um crescimento de 114 vezes, pois outras criptomoedas já obtiveram desempenho muito superior ao da Bitcoin, e com um *sharpe ratio* mais elevado (maior retorno com menor volatilidade). Consequentemente, investir no CMI20 permite ao investidor lucrar com o desempenho imprevisível das melhores criptomoedas, enquanto limita as perdas relativas as quedas de cada moeda isoladamente.

In [ ]:
# Index performance
plt.figure(figsize=(12,8))
# plt.plot(largest.index, np.log10(index_value));

plt.plot(pd.to_datetime(largest.index), cmi20, label='CMI20')

# plt.plot(pd.to_datetime(largest.index), closeDf.BTC / closeDf.BTC.iloc[0], label='BTC')

plt.title("Desempenho CMI20")
plt.ylabel("desempenho")
plt.xlabel("tempo")
plt.yscale('log')
plt.legend()
plt.grid(True, which='both');

In [ ]:
weekly_return = (cmi20.apply(lambda x: x[-1] / x[-8]).values[0] - 1) * 100
monthly_return = (cmi20.apply(lambda x: x[-1] / x[-29]).values[0] - 1) * 100
quaterly_return = (cmi20.apply(lambda x: x[-1] / x[-92]).values[0] - 1) * 100
halfyearly_return = (cmi20.apply(lambda x: x[-1] / x[-(92 * 2)]).values[0] - 1) * 100
yearly_return = (cmi20.apply(lambda x: x[-1] / x[-366]).values[0] - 1) * 100

msg =  'Retorno total:\n\n'
# msg += 'Semana:       %.2f %%\n' % weekly_return
msg += 'Mês:          %6.2f %%\n' % monthly_return
msg += 'Trimestre:    %6.2f %%\n' % quaterly_return
msg += 'Semestre:     %6.2f %%\n' % halfyearly_return
msg += 'Ano:          %6.2f %%\n' % yearly_return

print(msg)

O CMI20 é um dos instrumentos mais precisos para quantificar o desempenho do mercado de criptomoedas em geral. Este representa uma ferramenta muito útil para investidores, um *benchmark* para gerentes de investimentos, e um índice replicável para fundos de investimentos passívos. Resumindo, ele é o padrão da indústria para criptomoedas.

References:  
https://crypto20.com/en/  
https://thecryptosfund.com  
http://crix.hu-berlin.de  
http://www.bittwenty.com  
https://www.cci30.com  
https://www.bitwiseinvestments.com  
https://www.crypto30.com/index/  

*Este material tem fim educativo e não deve ser considerado como conselho de investimentos. O risco de perda de fundos, parcial ou total, é real. A Megáli não se responsabiliza por eventual perda de valor gerada pelo uso indevido deste ou de qualquer outro material divulgado por seus representantes.*